In [1]:
from __future__ import division
import os,time,cv2,scipy.io
import tensorflow as tf
# import tensorflow.contrib.slim as slim
import tf_slim as slim

import numpy as np
import matplotlib.pyplot as plt
# from discriminator import build_discriminator
import scipy.stats as st

In [2]:
# ENVIRONMENT
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2' 
import pathlib
from glob import glob

import warnings
warnings.filterwarnings("ignore")

# TENSORFLOW 2.0
import tensorflow as tf
print(tf.__version__)
print('GPU available:', tf.test.is_gpu_available())

from tensorflow.keras.models import Model
from tensorflow.python.keras.layers import Add, BatchNormalization, Conv2D, Dense, Flatten, Input, LeakyReLU, PReLU, Lambda, MaxPool2D
from tensorflow.keras.optimizers.schedules import PiecewiseConstantDecay
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import BinaryCrossentropy, MeanAbsoluteError, MeanSquaredError
binary_cross_entropy = BinaryCrossentropy()

from tensorflow.keras.metrics import Mean

import tensorflow.keras.backend as K
from tensorflow.python.data.experimental import AUTOTUNE

# ESSENTIAL 
import numpy as np
from sklearn.model_selection import train_test_split

# VISUALIZER
import matplotlib.pyplot as plt
plt.ioff()
from IPython import display

# UTILS
import time
from datetime import date
from tqdm import tqdm

2.1.0
Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.
GPU available: False


In [3]:
# Setting seed
tf.random.set_seed(27)
np.random.seed(27)

In [4]:
# Pre-trained VGG 
from tensorflow.python.keras.applications.vgg19 import VGG19
vgg_19 = VGG19(input_shape=(None, None, 3), weights='imagenet', include_top=False)

80142336/80134624 [==============================] - 12s 0us/step


In [6]:
vgg_mean = [103.939, 116.779, 123.68]

def _vgg(output_layer):
    return Model(vgg_19.input, vgg_19.layers[output_layer].output)

def vgg_52(input):
    input = tf.cast(input * 255., dtype=tf.float32)
    r, g, b = tf.split(input, 3, 3)
    bgr = tf.concat([b - vgg_mean[0],
                     g - vgg_mean[1],
                     r - vgg_mean[2]], axis=3)
    return _vgg(18)(bgr)

In [30]:
vgg_path=scipy.io.loadmat('./imagenet-vgg-verydeep-19.mat')
vgg_layers=vgg_path['layers'][0]

In [31]:
def get_weight_bias(vgg_layers,i):
    weights=vgg_layers[i][0][0][2][0][0]
    weights=tf.constant(weights)
    
    bias=vgg_layers[i][0][0][2][0][1]
    bias=tf.constant(np.reshape(bias,(bias.size)))
    return weights,bias

In [32]:
def build_net(ntype,nin,nwb=None,name=None):
    if ntype=='conv':
        return tf.nn.relu(tf.nn.conv2d(nin,nwb[0],strides=[1,1,1,1],padding='SAME',name=name)+  nwb[1])
    elif ntype=='pool':
        return tf.nn.avg_pool(nin,ksize=[1,2,2,1],strides=[1,2,2,1],padding='SAME')

In [33]:
def build_vgg19(input,reuse=False):
    net={}
    vgg_layers=vgg_path['layers'][0]
    
    net['input']= input - np.array([123.6800, 116.7790, 103.9390]).reshape((1,1,1,3))
    
    net['conv1_1']=build_net('conv',net['input'],get_weight_bias(vgg_layers,0),name='vgg_conv1_1')
    net['conv1_2']=build_net('conv',net['conv1_1'],get_weight_bias(vgg_layers,2),name='vgg_conv1_2')
    net['pool1']=build_net('pool',net['conv1_2'])
    net['conv2_1']=build_net('conv',net['pool1'],get_weight_bias(vgg_layers,5),name='vgg_conv2_1')
    net['conv2_2']=build_net('conv',net['conv2_1'],get_weight_bias(vgg_layers,7),name='vgg_conv2_2')
    net['pool2']=build_net('pool',net['conv2_2'])
    net['conv3_1']=build_net('conv',net['pool2'],get_weight_bias(vgg_layers,10),name='vgg_conv3_1')
    net['conv3_2']=build_net('conv',net['conv3_1'],get_weight_bias(vgg_layers,12),name='vgg_conv3_2')
    net['conv3_3']=build_net('conv',net['conv3_2'],get_weight_bias(vgg_layers,14),name='vgg_conv3_3')
    net['conv3_4']=build_net('conv',net['conv3_3'],get_weight_bias(vgg_layers,16),name='vgg_conv3_4')
    net['pool3']=build_net('pool',net['conv3_4'])
    net['conv4_1']=build_net('conv',net['pool3'],get_weight_bias(vgg_layers,19),name='vgg_conv4_1')
    net['conv4_2']=build_net('conv',net['conv4_1'],get_weight_bias(vgg_layers,21),name='vgg_conv4_2')
    net['conv4_3']=build_net('conv',net['conv4_2'],get_weight_bias(vgg_layers,23),name='vgg_conv4_3')
    net['conv4_4']=build_net('conv',net['conv4_3'],get_weight_bias(vgg_layers,25),name='vgg_conv4_4')
    net['pool4']=build_net('pool',net['conv4_4'])
    net['conv5_1']=build_net('conv',net['pool4'],get_weight_bias(vgg_layers,28),name='vgg_conv5_1')
    net['conv5_2']=build_net('conv',net['conv5_1'],get_weight_bias(vgg_layers,30),name='vgg_conv5_2')
    return net

In [43]:
# Generator (a.k.a Reflection Removal Model)
# initializer = tf.keras.initializers.Identity(gain= 1.0)
initializer = tf.initializers.he_normal(seed=None)


def activation_normalizer(layer):
    layer = tf.keras.layers.LeakyReLU(0.2)(layer)
    layer = tf.keras.layers.BatchNormalization()(layer)
    return layer
    
    
def build_gen(input_shape):
    
    x_input = tf.keras.layers.Input(shape = input_shape)
    
    vgg19_features = vgg_52(x_input) 
    
    gen = tf.keras.layers.Conv2D(filters= 64, kernel_size= [1,1], dilation_rate= 1,padding= 'same', kernel_initializer= initializer)(vgg19_features)
    gen = activation_normalizer(gen)
    gen = tf.keras.layers.Conv2D(64, kernel_size= [3,3], dilation_rate= 1, padding= 'same', kernel_initializer= initializer)(gen)
    gen = activation_normalizer(gen)
    gen = tf.keras.layers.Conv2D(64, kernel_size= [3,3], dilation_rate= 2, padding= 'same', kernel_initializer= initializer)(gen)
    gen = activation_normalizer(gen)
    gen = tf.keras.layers.Conv2D(64, kernel_size= [3,3], dilation_rate= 4, padding= 'same', kernel_initializer= initializer)(gen)
    gen = activation_normalizer(gen)
    gen = tf.keras.layers.Conv2D(64, kernel_size= [3,3], dilation_rate= 8, padding= 'same', kernel_initializer= initializer)(gen)
    gen = activation_normalizer(gen)
    gen = tf.keras.layers.Conv2D(64, kernel_size= [3,3], dilation_rate= 16, padding= 'same', kernel_initializer= initializer)(gen)
    gen = activation_normalizer(gen)
    gen = tf.keras.layers.Conv2D(64, kernel_size= [3,3], dilation_rate= 32, padding= 'same', kernel_initializer= initializer)(gen)
    gen = activation_normalizer(gen)
    gen = tf.keras.layers.Conv2D(64, kernel_size= [3,3], dilation_rate= 64, padding= 'same', kernel_initializer= initializer)(gen)
    gen = activation_normalizer(gen)
    gen = tf.keras.layers.Conv2D(64, kernel_size= [3,3], dilation_rate= 1, padding= 'same', kernel_initializer= initializer)(gen)
    gen = activation_normalizer(gen)
    # last layer
    gen = tf.keras.layers.Conv2D(6, kernel_size= [1,1], dilation_rate= 1, padding= 'same')(gen)
    gen = Model(x_input, gen)
    return gen

In [44]:
generator = build_gen((None, None, 3))

In [45]:
generator.summary()

Model: "model_6"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_11 (InputLayer)           [(None, None, None,  0                                            
__________________________________________________________________________________________________
tf_op_layer_mul_1 (TensorFlowOp [(None, None, None,  0           input_11[0][0]                   
__________________________________________________________________________________________________
tf_op_layer_split_1 (TensorFlow [(None, None, None,  0           tf_op_layer_mul_1[0][0]          
__________________________________________________________________________________________________
tf_op_layer_sub_3 (TensorFlowOp [(None, None, None,  0           tf_op_layer_split_1[0][2]        
____________________________________________________________________________________________